# Merging Airbnb and Subway Station Datasets

This notebook merges Airbnb listings with NYC subway station data to create a new dataset containing:
- Airbnb information (name, location, price)
- Nearest subway station details
- Distance metrics and station counts

**NYC Center Coordinates:** (40.75600, -73.98470)

## Step 1: Import Libraries and Set Up GPU Support

In [8]:
import pandas as pd
import numpy as np
from math import radians

# Detect GPU backend
GPU_BACKEND = None

# Check for Apple Silicon (MPS) via PyTorch
try:
    import torch
    if torch.backends.mps.is_available():
        GPU_BACKEND = "mps"
        device = torch.device("mps")
        print(f"Apple Silicon GPU detected (M3 Max)")
        print(f"Using PyTorch MPS backend for GPU acceleration")
    elif torch.cuda.is_available():
        GPU_BACKEND = "cuda"
        device = torch.device("cuda")
        print(f"NVIDIA GPU detected, using CUDA")
    else:
        print("PyTorch available but no GPU backend detected")
except ImportError:
    print("PyTorch not installed")

# Fallback to CuPy for NVIDIA GPUs if PyTorch not available
if GPU_BACKEND is None:
    try:
        import cupy as cp
        GPU_BACKEND = "cupy"
        print("GPU acceleration available via CuPy")
    except ImportError:
        pass

if GPU_BACKEND is None:
    print("No GPU acceleration available, using NumPy (CPU)")

# NYC Center coordinates
NYC_CENTER_LAT = 40.75600
NYC_CENTER_LON = -73.98470

Apple Silicon GPU detected (M3 Max)
Using PyTorch MPS backend for GPU acceleration


## Step 2: Load Airbnb Data

In [9]:
# Load Airbnb data
airbnb_df = pd.read_csv('data/AB_NYC_2019.csv')

# Select relevant columns
airbnb_df = airbnb_df[['name', 'latitude', 'longitude', 'price']].copy()
airbnb_df.columns = ['airbnb_name', 'airbnb_latitude', 'airbnb_longitude', 'airbnb_price']

# Drop rows with missing coordinates
airbnb_df = airbnb_df.dropna(subset=['airbnb_latitude', 'airbnb_longitude'])

print(f"Loaded {len(airbnb_df)} Airbnb listings")
airbnb_df.head()

Loaded 48895 Airbnb listings


,airbnb_name,airbnb_latitude,airbnb_longitude,airbnb_price
0,Clean & quiet apt home by the park,40.64749,-73.97237,149
1,Skylit Midtown Castle,40.75362,-73.98377,225
2,THE VILLAGE OF HARLEM....NEW YORK !,40.80902,-73.94190,150
3,Cozy Entire Floor of Brownstone,40.68514,-73.95976,89
4,Entire Apt: Spacious Studio/Loft by central park,40.79851,-73.94399,80


## Step 3: Extract Unique Subway Stations from MTA Data

Since the MTA file is very large (7+ GB with hourly ridership data), we'll read it in chunks and extract unique stations.

In [10]:
# Read MTA data in chunks and extract unique stations
# We only need station_complex (name), latitude, and longitude

unique_stations = set()
station_coords = {}

# Read in chunks to handle large file
chunk_size = 500000  # 500K rows at a time
print("Extracting unique subway stations from MTA data...")

for i, chunk in enumerate(pd.read_csv('data/MTA_Subway_Hourly_Ridership.csv', 
                                       usecols=['station_complex', 'latitude', 'longitude'],
                                       chunksize=chunk_size)):
    # Get unique stations from this chunk
    for _, row in chunk.drop_duplicates(subset=['station_complex']).iterrows():
        station_name = row['station_complex']
        if station_name not in station_coords:
            station_coords[station_name] = (row['latitude'], row['longitude'])
    
    if (i + 1) % 20 == 0:
        print(f"Processed {(i + 1) * chunk_size:,} rows, found {len(station_coords)} unique stations...")

print(f"\nTotal unique subway stations: {len(station_coords)}")

Extracting unique subway stations from MTA data...
Processed 10,000,000 rows, found 428 unique stations...
Processed 20,000,000 rows, found 428 unique stations...
Processed 30,000,000 rows, found 428 unique stations...
Processed 40,000,000 rows, found 428 unique stations...
Processed 50,000,000 rows, found 428 unique stations...

Total unique subway stations: 428


In [11]:
# Create DataFrame of unique stations
stations_df = pd.DataFrame([
    {'station_name': name, 'station_latitude': coords[0], 'station_longitude': coords[1]}
    for name, coords in station_coords.items()
])

# Drop any stations with missing coordinates
stations_df = stations_df.dropna()

print(f"Subway stations DataFrame: {len(stations_df)} stations")
stations_df.head()

Subway stations DataFrame: 428 stations


,station_name,station_latitude,station_longitude
0,Liberty Av (C),40.674540,-73.896545
1,Bushwick Av-Aberdeen St (L),40.682830,-73.905250
2,"High St (A,C)",40.699337,-73.990530
3,Livonia Av (L),40.664040,-73.900570
4,Junius St (3),40.663513,-73.902450


## Step 4: Define Haversine Distance Function (GPU-Accelerated)

The Haversine formula calculates the great-circle distance between two points on a sphere given their latitudes and longitudes.

In [12]:
def haversine_distance_vectorized(lat1, lon1, lat2, lon2):
    """
    Calculate Haversine distance between two sets of coordinates.
    Returns distance in meters.
    
    Automatically uses the best available backend:
    - PyTorch MPS for Apple Silicon (M3 Max)
    - PyTorch CUDA for NVIDIA GPUs
    - CuPy for NVIDIA GPUs (fallback)
    - NumPy for CPU
    
    Parameters:
    - lat1, lon1: Arrays of latitudes/longitudes for first set of points
    - lat2, lon2: Arrays of latitudes/longitudes for second set of points
    
    Returns:
    - Distance matrix of shape (len(lat1), len(lat2)) in meters
    """
    # Earth's radius in meters
    R = 6371000
    
    if GPU_BACKEND in ["mps", "cuda"]:
        # Use PyTorch for Apple Silicon MPS or NVIDIA CUDA
        # MPS only supports float32, CUDA supports float64
        dtype = torch.float32 if GPU_BACKEND == "mps" else torch.float64
        
        lat1_t = torch.tensor(lat1, dtype=dtype, device=device).reshape(-1, 1)
        lon1_t = torch.tensor(lon1, dtype=dtype, device=device).reshape(-1, 1)
        lat2_t = torch.tensor(lat2, dtype=dtype, device=device).reshape(1, -1)
        lon2_t = torch.tensor(lon2, dtype=dtype, device=device).reshape(1, -1)
        
        # Convert to radians
        lat1_rad = torch.deg2rad(lat1_t)
        lon1_rad = torch.deg2rad(lon1_t)
        lat2_rad = torch.deg2rad(lat2_t)
        lon2_rad = torch.deg2rad(lon2_t)
        
        # Haversine formula
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad
        
        a = torch.sin(dlat / 2) ** 2 + torch.cos(lat1_rad) * torch.cos(lat2_rad) * torch.sin(dlon / 2) ** 2
        c = 2 * torch.arcsin(torch.sqrt(a))
        
        distance = R * c
        return distance.cpu().numpy()
    
    elif GPU_BACKEND == "cupy":
        # Use CuPy for NVIDIA GPUs
        lat1_rad = cp.radians(cp.asarray(lat1).reshape(-1, 1))
        lon1_rad = cp.radians(cp.asarray(lon1).reshape(-1, 1))
        lat2_rad = cp.radians(cp.asarray(lat2).reshape(1, -1))
        lon2_rad = cp.radians(cp.asarray(lon2).reshape(1, -1))
        
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad
        
        a = cp.sin(dlat / 2) ** 2 + cp.cos(lat1_rad) * cp.cos(lat2_rad) * cp.sin(dlon / 2) ** 2
        c = 2 * cp.arcsin(cp.sqrt(a))
        
        return cp.asnumpy(R * c)
    
    else:
        # Fallback to NumPy (CPU)
        lat1_rad = np.radians(np.asarray(lat1).reshape(-1, 1))
        lon1_rad = np.radians(np.asarray(lon1).reshape(-1, 1))
        lat2_rad = np.radians(np.asarray(lat2).reshape(1, -1))
        lon2_rad = np.radians(np.asarray(lon2).reshape(1, -1))
        
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad
        
        a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
        c = 2 * np.arcsin(np.sqrt(a))
        
        return R * c

def haversine_single(lat1, lon1, lat2, lon2):
    """Calculate Haversine distance between two single points in meters."""
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return 6371000 * c

print(f"Haversine distance functions defined (using: {GPU_BACKEND or 'numpy'})")

Haversine distance functions defined (using: mps)


## Step 5: Calculate Distance Matrix (Airbnb to All Stations)

This step calculates the distance from each Airbnb listing to every subway station using GPU acceleration if available.

In [13]:
%%time

# Extract coordinate arrays
airbnb_lats = airbnb_df['airbnb_latitude'].values
airbnb_lons = airbnb_df['airbnb_longitude'].values
station_lats = stations_df['station_latitude'].values
station_lons = stations_df['station_longitude'].values

print(f"Calculating distances: {len(airbnb_lats)} Airbnbs x {len(station_lats)} stations")
print(f"Using backend: {GPU_BACKEND or 'numpy (CPU)'}")

# Calculate distance matrix (Airbnbs x Stations)
# Shape: (num_airbnbs, num_stations)
distance_matrix = haversine_distance_vectorized(
    airbnb_lats, airbnb_lons,
    station_lats, station_lons
)

print(f"Distance matrix shape: {distance_matrix.shape}")

Calculating distances: 48895 Airbnbs x 428 stations
Using backend: mps
Distance matrix shape: (48895, 428)
CPU times: user 1.06 s, sys: 2.44 s, total: 3.49 s
Wall time: 521 ms


## Step 6: Find Nearest Station and Count Stations Within 500m

In [14]:
# Find nearest station for each Airbnb
nearest_station_idx = np.argmin(distance_matrix, axis=1)
nearest_station_distance = np.min(distance_matrix, axis=1)

# Get nearest station details
nearest_station_names = stations_df.iloc[nearest_station_idx]['station_name'].values
nearest_station_lats = stations_df.iloc[nearest_station_idx]['station_latitude'].values
nearest_station_lons = stations_df.iloc[nearest_station_idx]['station_longitude'].values

# Count stations within 500 meters
stations_within_500m = np.sum(distance_matrix <= 500, axis=1)

print(f"Average distance to nearest station: {nearest_station_distance.mean():.2f} meters")
print(f"Average stations within 500m: {stations_within_500m.mean():.2f}")

Average distance to nearest station: 455.85 meters
Average stations within 500m: 1.51


## Step 7: Calculate Distance from Nearest Station to NYC Center

In [15]:
# Calculate distance from each nearest station to NYC center
# Using vectorized calculation for efficiency
nearest_station_to_nyc_center = np.array([
    haversine_single(lat, lon, NYC_CENTER_LAT, NYC_CENTER_LON)
    for lat, lon in zip(nearest_station_lats, nearest_station_lons)
])

print(f"Average distance from nearest station to NYC center: {nearest_station_to_nyc_center.mean():.2f} meters")

Average distance from nearest station to NYC center: 6909.94 meters


## Step 8: Create Final Merged Dataset

In [16]:
# Create the final merged DataFrame
merged_df = pd.DataFrame({
    'airbnb_name': airbnb_df['airbnb_name'].values,
    'airbnb_latitude': airbnb_df['airbnb_latitude'].values,
    'airbnb_longitude': airbnb_df['airbnb_longitude'].values,
    'airbnb_price': airbnb_df['airbnb_price'].values,
    'distance_to_nearest_station_m': nearest_station_distance,
    'nearest_station_name': nearest_station_names,
    'nearest_station_latitude': nearest_station_lats,
    'nearest_station_longitude': nearest_station_lons,
    'stations_within_500m': stations_within_500m,
    'nearest_station_to_nyc_center_m': nearest_station_to_nyc_center
})

print(f"Final merged dataset: {len(merged_df)} rows, {len(merged_df.columns)} columns")
merged_df.head(10)

Final merged dataset: 48895 rows, 10 columns


,airbnb_name,airbnb_latitude,airbnb_longitude,airbnb_price,distance_to_nearest_station_m,nearest_station_name,nearest_station_latitude,nearest_station_longitude,stations_within_500m,nearest_station_to_nyc_center_m
0,Clean & quiet apt home by the park,40.64749,-73.97237,149,465.851501,"Fort Hamilton Pkwy (F,G)",40.650784,-73.975780,1,11723.623487
1,Skylit Midtown Castle,40.75362,-73.98377,225,165.939743,"Bryant Pk (B,D,F,M)/5 Av (7)",40.754223,-73.981964,2,303.567132
2,THE VILLAGE OF HARLEM....NEW YORK !,40.80902,-73.94190,150,333.582275,"125 St (2,3)",40.807755,-73.945496,1,6634.339736
3,Cozy Entire Floor of Brownstone,40.68514,-73.95976,89,416.244415,Classon Av (G),40.688873,-73.960070,2,7747.401786
4,Entire Apt: Spacious Studio/Loft by central park,40.79851,-73.94399,80,199.920410,116 St (6),40.798630,-73.941620,2,5968.944848
5,Large Cozy 1 BR Apartment In Midtown East,40.74767,-73.97500,200,482.519592,"Grand Central-42 St (S,4,5,6,7)",40.751778,-73.976845,1,811.278447
6,BlissArtsSpace!,40.68688,-73.95596,60,369.033020,Bedford-Nostrand Avs (G),40.689630,-73.953520,2,7833.821603
7,Large Furnished Room Near B'way,40.76489,-73.98493,79,284.518311,"50 St (C,E)",40.762455,-73.985985,5,725.877277
8,Cozy Clean Guest Room - Family Apt,40.80178,-73.96723,79,245.157455,Cathedral Pkwy (110 St) (1),40.803967,-73.966850,2,5541.399014
9,Cute & Cozy Lower East Side 1 bdrm,40.71344,-73.99037,150,34.664215,East Broadway (F),40.713715,-73.990170,1,4724.411630


In [17]:
# Display summary statistics
merged_df.describe()

,airbnb_latitude,airbnb_longitude,airbnb_price,distance_to_nearest_station_m,nearest_station_latitude,nearest_station_longitude,stations_within_500m,nearest_station_to_nyc_center_m
count,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000
mean,40.728949,-73.952170,152.720687,455.854950,40.729133,-73.952576,1.511975,6909.942222
std,0.054530,0.046157,240.154170,734.639099,0.053983,0.043262,1.340554,4221.255674
min,40.499790,-74.244420,0.000000,0.689171,40.576126,-74.074840,0.000000,226.227131
25%,40.690100,-73.983070,69.000000,206.817970,40.689630,-73.982210,1.000000,3688.644513
50%,40.723070,-73.955680,106.000000,318.709167,40.723400,-73.956850,1.000000,6260.419038
75%,40.763115,-73.936275,175.000000,470.527359,40.762455,-73.936470,2.000000,9297.148927
max,40.913060,-73.712990,10000.000000,20717.220703,40.903126,-73.755400,9.000000,25681.865358


## Step 9: Save Merged Dataset

In [18]:
# Save to CSV
output_path = 'data/airbnb_subway_merged.csv'
merged_df.to_csv(output_path, index=False)
print(f"Merged dataset saved to: {output_path}")

Merged dataset saved to: data/airbnb_subway_merged.csv


In [19]:
# Verify output columns
print("Final Dataset Columns:")
print("-" * 50)
for col in merged_df.columns:
    print(f"- {col}: {merged_df[col].dtype}")
print("-" * 50)
print(f"\nTotal rows: {len(merged_df)}")

Final Dataset Columns:
--------------------------------------------------
- airbnb_name: object
- airbnb_latitude: float64
- airbnb_longitude: float64
- airbnb_price: int64
- distance_to_nearest_station_m: float32
- nearest_station_name: object
- nearest_station_latitude: float64
- nearest_station_longitude: float64
- stations_within_500m: int64
- nearest_station_to_nyc_center_m: float64
--------------------------------------------------

Total rows: 48895
